# Install and import libraries

In [ ]:
#!pip install numpy
#!pip install simpy
#!pip install matplotlib

In [ ]:
import simpy
import numpy as np
import numpy.random as random

# Parameters setting

In [1]:
MAXSIMTIME = 50000
VERBOSE = False
LAMBDA = 3.8
MU = 8.0
POPULATION = 50000000
SERVICE_DISCIPLINE = 'FCFS'
LOGGED = True
PLOTTED = True

# Discrete-Events Simulation model

## The definition of a job

Properties of a jobs are:
* Job execution time
* Job arrival time

In [2]:
class Job:
    def __init__(self, name, arrtime, duration):
        self.name = name
        self.arrtime = arrtime
        self.duration = duration
        
    def __str__(self):
        return '%s at %d, length %d' %(self.name, self.arrtime, self.duration)

## Discipline
For this simulation, we will use the FCFS discipline for our model.

In [3]:
def FCFS(job):
    return job.duration

## The definition of a server
There are 2 arguments needed for a server
1. env: SimPy environment
1. Queue discipline: First come, first served

In [ ]:
class SubServer:
    def __init__(self, env, strat = 'FCFS'):
        self.env = env
        self.strat = strat
        self.Jobs = list(())
        self.serversleeping = None
        '''statistics'''
        self.waitingTime = 0
        self.idleTime = 0
        self.jobsDone = 0
        '''register a new server process'''
        env.process(self.serve())
        
    def serve(self):
        while True:
            '''do nothing, just change server to idle
               and then yield a wait event which takes infinite time'''
            if len(self.Jobs == 0):
                self.serversleeping = env.process(self.waiting(self.env))
                t1 = self.env.now
                yield self.serversleeping
                '''accumulate the server idle time'''
                seld.idleTime += self.env.now - t1
            else:
                '''get the first job to be served'''
                if self.strat == 'SJF':
                    self.Jobs.sort( key = SJF )
                    j = self.Jobs.pop( 0 )
                else: # FIFO by default
                    j = self.Jobs.pop( 0 )
                if LOGGED:
                    qlog.write( '%.4f\t%d\t%d\n' 
                        % (self.env.now, 1 if len(self.Jobs)>0 else 0, len(self.Jobs)) )

                ''' sum up the waiting time'''
                self.waitingTime += self.env.now - j.arrtime
                ''' yield an event for the job finish'''
                yield self.env.timeout( j.duration )
                ''' sum up the jobs done '''
                self.jobsDone += 1

    def waiting(self, env):
        try:
            if VERBOSE:
                print( 'Server is idle at %.2f' % self.env.now )
            yield self.env.timeout( MAXSIMTIME )
        except simpy.Interrupt as i:
            if VERBOSE:
                 print('Server waken up and works at %.2f' % self.env.now )